# 75.26 Modelos y Simulación

# Trabajo Práctico 2

## Grupo 2

### Alumnos:

- Duzac, Emilia
- Belletti, Gabriel Ignacio
- Vazquez, Nicolás Alberto
- Cohen, Martín
-
### Generador de números aleatorios elegido: [MIXMAX](http://arxiv.org/pdf/1403.5355)

## Ejercicio 2

### método asignado en la planilla de grupos: SIMPY

Una entidad financiera está analizando cambiar uno de sus cajeros automáticos por uno con la funcionalidad de reciclador
de billetes (si un cliente realiza un depósito de efectivo, ese efectivo estará disponible para que lo pueda retirar otro cliente).
El cajero automático actual, que se quiere reemplazar, es de tipo estándar. No recicla los billetes, sino que posee dos
compartimentos, uno con dinero para entregar y otro donde guarda el dinero que se deposita.
Esto presenta los siguientes problemas:

- Si no tiene más efectivo para entregar en el compartimento de retiro de dinero, no entrega efectivo, aunque tenga en el compartimento de depósitos.
- Si el compartimento que recibe los depósitos se completa, no puede recibir más.

La entidad conoce que con el cajero actual el 25% de los clientes se retiraba sin poder extraer dinero, y sólo acepta cambiarlo si este porcentaje disminuye.

Por simplicidad suponer que el cajero sólo expende billetes de una denominación, tiene una capacidad máxima de 2000
billetes, y comienza el día con su carga completa.

En el relevamiento que realizó la entidad diferenció a sus clientes en dos grupos:

- Grupo 1: Sólo retiran efectivo, y corresponden al 75% de los clientes que arriban El tiempo que utilizan el cajero se puede modelar con una distribución exponencial de media 90 segundos. Cada cliente de este grupo, extrae una cantidad de billetes que sigue una distribución uniforme $[3,50]$.
- Grupo 2: Sólo realizan depósitos, son el 25% de los clientes totales. Utilizando el cajero un tiempo que siguen una distribución exponencial de media 5 minutos ($5min = 300seg$). Cada cliente de este grupo, deposita una cantidad de billetes que se puede modelar utilizando una distribución uniforme $[10,110]$.

El área a cargo de modelar el problema, propuso un tiempo de arribo de clientes que sigue una distribución de
probabilidades Exponencial con media = 180 seg

### Se pide:

1. Simular, utilizando el generador de números al azar desarrollado en el tp 1, 100 días completos de 24 hrs,
independientes. (utilizando distintas semillas en el generador).
2. Graficar la cantidad media de billetes disponibles en el cajero luego de cada transacción.
3. Graficar la distribución de los tiempos que los clientes demoran en el sistema (espera + utilización del cajero).
4. Calcular el tiempo medio y el desvío estandar del tiempo que los clientes demoran en el Sistema.
5. Proponer otras frecuencias de arribos de clientes y comprar los resultados obtenidos.

### Consideraciones/Suposiciones

- El cajero solo admite billetes de una denominacion por lo que se hablara de cantidad de billetes en lugar del monto representado por estos.

### Imports

In [1]:
import numpy as np
# import matplotlib.pyplot as plt
import simpy

In [2]:
from MixMax import MixmaxRNG

### Constantes

In [3]:
# cantidad de dias simulados
TOTAL_SIMULATIONS = 100
# tiempo total del dia para recarga de efectivo (cada dia se recarga el efectivo) 3600 * 24 = 86 400
DAY_TIME = 86400.0
# arribo de clientes 1 cliente cada 180s (de media) equivale a 1/180 clientes/s
ARRIVAL_CLIENT = 1.0/180.0

# GRUPO 1 retiro de efectivo
# proporcion de grupo 1
PROPORTION_GROUP_1 = 0.75
# intervalo de valores posibles de extraccion (cantidad de billetes que se extraen)
EXTRACTION_INTERVAL = (3, 50)
# tiempo de uso del cajero
TIME_LAMBDA_GROUP_1 = 1/90

# GRUPO 2 ingreso de efectivo
# intervalo de valores posibles de deposito (cantidad de billetes que se depositan)
DEPOSIT_INTERVAL = (10, 110)
# tiempo de uso del cajero
TIME_LAMBDA_GROUP_2 = 1/300

# capacidad maxima de billetes del cajero
MAX_AMOUNT_OF_BILLS = 2000

# semillas
# semilla de arrivos
SEED_ARRIVALS = 1235
# semilla base de clientes
SEED_CLIENT_TIPE = 785

# cantidad de cajeros disponibles
NUMBER_OF_CASHIERS = 1

# proporcion de extracciones fallidas antes
EXTRACTION_FAILURE_OLD = 0.25

### Codigo

In [10]:
class DailyCustomerEvent:
    def __init__(self):
        self.time_delay = []
        self.n_bills = []
        self.n_extraction_failure = 0
        self.extraction_attempts = 0

    def extraction(self, rest, time, is_failure):
        self.extraction_attempts += 1
        if is_failure:
            self.n_extraction_failure += 1
        self.time_delay.append(time)
        self.n_bills.append(rest)

    def income(self, rest, time):
        self.time_delay.append(time)
        self.n_bills.append(rest)

    def median_bills(self):
        return np.median(self.n_bills)

    def time_median_and_std(self):
        median = np.median(self.time_delay)
        std = np.std(self.time_delay)
        return median, std

    def extraction_failure_accepted(self, extration_failure_old=EXTRACTION_FAILURE_OLD):
        extraction_failure_proportion = self.n_extraction_failure / self.extraction_attempts
        return extration_failure_old < extraction_failure_proportion


In [5]:
def exponencial(val_p, value_lambda):
    return -(np.log(1 - val_p)) / value_lambda

"""returns a list with the time of arrivals (not accumulated), up to the given maximum time."""
def generate_arrivals(seed, total_seconds, value_lambda):
    result = []
    total = 0.0
    # generador de numeros aleatorios
    random = MixmaxRNG(seed)
    while True:
        # nuevo arribo en segundos
        exp = exponencial(random.generate_number(), value_lambda)
        # tiempo acumulado de arribo
        total += exp
        # caso donde se pasa del tiempo maximo corta el bucle
        if total > total_seconds:
            break
        # se agrega el tiempo entre arrivos (NO acumulado)
        result.append(exp)
    return result

In [6]:
"""returns true if, according to the given value, it belongs to group 1."""
def is_group_1(p_val):
    return p_val < PROPORTION_GROUP_1

"""client withdrawing money."""
def client_group_1(env, cashier, random, res_cashier, arrival_time, statistics, debug):
    # generan el valor de la cantidad de billetes que usaran (extraer)
    extraction = int(round(random.generate_number(interval=EXTRACTION_INTERVAL), 0))
    # tiempo que nesesito para realizar la operacion
    time = exponencial(random.generate_number(), TIME_LAMBDA_GROUP_1)
    with res_cashier.request() as req:
        yield req
        if debug:
            print(f'{env.now:7.3f} s: Extraction of {extraction} begins.')
        # caso donde hay dinero para extraer
        permission = cashier.level >= extraction
        if permission:
            cashier.get(extraction)
        # consume tiempo manteniendo el recurso para si
        yield env.timeout(delay=time)
        if debug:
            print(f'{env.now:7.3f} s: Correct extraction {permission}.', )
        statistics.extraction(cashier.level, env.now - arrival_time, permission)
    return

"""customer depositing money."""
def client_group_2(env, cashier, random, res_cashier, arrival_time, statistics, debug):
    # generan el valor de la cantidad de billetes que usaran (depositar)
    deposit = int(round(random.generate_number(interval=DEPOSIT_INTERVAL), 0))
    # tiempo que nesesito para realizar la operacion
    time = exponencial(random.generate_number(), TIME_LAMBDA_GROUP_2)
    with res_cashier.request() as req:
        yield req
        if debug:
            print(f'{env.now:7.3f} s: Deposit of {deposit} begins.')
        # caso donde hay espacio para depositar dinero
        permission = cashier.level + deposit <= MAX_AMOUNT_OF_BILLS
        if permission:
            cashier.put(deposit)
        # consume tiempo manteniendo el recurso para si
        yield env.timeout(delay=time)
        if debug:
            print(f'{env.now:7.3f} s: Correct deposit {permission}.')
        statistics.income(cashier.level, env.now - arrival_time)
    return

""""""
def client(env, list_arrivals_of_clients, seed, cashier, res_cashier, statistics, debug):
    random = MixmaxRNG(seed)
    # para cada arrivo se clasifica en los dos grupos
    if debug:
        print(f'{env.now:7.3f} s: Clients arrivals begins.')
    for delay in list_arrivals_of_clients:
        # pasa un tiempo antes del evento
        yield env.timeout(delay=delay)
        arrival_time = env.now
        if debug:
            print(f'{arrival_time:7.3f} s: Arrived new client.')
        # se toma un numero al azar (uniforme) para saber en que grupo caen
        p_val = random.generate_number()
        if is_group_1(p_val):
            env.process(client_group_1(env, cashier, random, res_cashier, arrival_time, statistics, debug))
        else:
            env.process(client_group_2(env, cashier, random, res_cashier, arrival_time, statistics, debug))
    if debug:
        print(f'{env.now:7.3f} s: Clients arrivals finish.')

In [7]:
# def daily_recharge(env, cashier, res_cashier, cashier_reload_time, debug):
#     # bucle eterno
#     while True:
#         with res_cashier.request() as req:
#             yield req
#             diff = cashier.capacity - cashier.level
#             if debug:
#                 print(f'{env.now:7.3f} s: Recharge cashier, total = {cashier.level}.')
#             if diff > 0:
#                 cashier.put(diff)
#                 if debug:
#                     print(f'{env.now:7.3f} s: Recharge complete, total = {cashier.level}.')
#             elif debug:
#                 print(f'{env.now:7.3f} s: No recharge requiere.')
#         yield env.timeout(cashier_reload_time)  # Chequeo

In [13]:
def daily_simulation(seed_arrivals=SEED_ARRIVALS, seed_clients=SEED_CLIENT_TIPE, arrival_client=ARRIVAL_CLIENT, total_time=DAY_TIME, debug=False):
    # estadisticas de la simulacion
    statistics = DailyCustomerEvent()
    # creacion del entorno simpy
    env = simpy.Environment()
    # creacion de recurso de cajero (contiene billetes) (inicia con la maxima cantidad de billetes)
    cashier = simpy.Container(env, MAX_AMOUNT_OF_BILLS, init=MAX_AMOUNT_OF_BILLS)
    # cantidad de recurso cajero disponible
    res_cashier = simpy.Resource(env, NUMBER_OF_CASHIERS)
    # llegada de clientes
    arrivals_of_clients = generate_arrivals(seed_arrivals, total_time, arrival_client)
    # procesado de clientes
    proc = env.process(client(env, arrivals_of_clients, seed_clients, cashier, res_cashier, statistics, debug))
    env.run(until=proc)
    return statistics

stats = daily_simulation(total_time=600.0, debug=True, arrival_client=1/60)
print("Median n bills: ", stats.median_bills(), "Median;std of time: ", stats.time_median_and_std(), "Failure extraction < 0.25: ", stats.extraction_failure_accepted())

  0.000 s: Clients arrivals begins.
 40.164 s: Arrived new client.
 40.164 s: Extraction of 48 begins.
 65.897 s: Arrived new client.
103.944 s: Arrived new client.
123.355 s: Correct extraction True.
123.355 s: Extraction of 28 begins.
175.393 s: Arrived new client.
192.105 s: Correct extraction True.
192.105 s: Extraction of 7 begins.
220.546 s: Correct extraction True.
220.546 s: Extraction of 8 begins.
303.350 s: Arrived new client.
307.565 s: Arrived new client.
349.824 s: Arrived new client.
382.132 s: Arrived new client.
411.433 s: Correct extraction True.
411.433 s: Extraction of 46 begins.
427.199 s: Arrived new client.
495.180 s: Arrived new client.
499.079 s: Arrived new client.
499.079 s: Clients arrivals finish.
Median n bills:  1920.5 Median;std of time:  (121.40492082637331, 57.418650120008756) Failure extraction < 0.25:  True


In [9]:
def simulaction():
    return